# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.



 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [ ]:
after each change commit

## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [615]:
import mysql.connector 
import config


In [616]:
from mysql.connector import errorcode

In [617]:
## Connect to DB server on AWS
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()

In [618]:
#drop_table = """DROP TABLE reviews;"""

In [619]:
# cnx = mysql.connector.connect(
#     host = config.host,
#     user = config.user,
#     passwd = config.password,
#     database = 'yelp'
# )
# cursor = cnx.cursor()

# cnx.commit()





In [620]:
# cursor.execute(drop_table)

In [621]:
# drop_table = """DROP Table reviews;"""

In [622]:
## Create new DB
db_name = 'yelp'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [623]:
# Create a table for the Businesses
DB_NAME = 'yelp'

TABLES = {} #use dictionary to save diff create statements for each table
TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    
    "  id varchar(50) NOT NULL PRIMARY KEY," #index automatically created
    "  name varchar(255) NOT NULL,"
    "  rating int(11) NOT NULL,"
    "  price varchar(5) NOT NULL"
    ") ENGINE=InnoDB")



In [624]:
TABLES['businesses']

'CREATE TABLE businesses (  id varchar(50) NOT NULL PRIMARY KEY,  name varchar(255) NOT NULL,  rating int(11) NOT NULL,  price varchar(5) NOT NULL) ENGINE=InnoDB'

In [625]:
# Create a table for the reviews
TABLES['reviews'] = """
    CREATE TABLE review (

      id varchar(50) NOT NULL,
      name varchar(255) NOT NULL,
      rating_review int(1) NOT NULL,
      
      datetime_of_review date NOT NULL,
      review varchar(500) NOT NULL,
      PRIMARY KEY (id),
      FOREIGN KEY (id) REFERENCES businesses (id)
    ) ENGINE=InnoDB
"""







#       review_primary_key (Bus_primary_key,bus_name) UNIQUE NOT NULL AUTO_INCREMENT PRIMARY KEY,


In [626]:
TABLES['reviews']

'\n    CREATE TABLE review (\n\n      id varchar(50) NOT NULL,\n      name varchar(255) NOT NULL,\n      rating_review int(1) NOT NULL,\n      \n      datetime_of_review date NOT NULL,\n      review varchar(500) NOT NULL,\n      PRIMARY KEY (id),\n      FOREIGN KEY (id) REFERENCES businesses (id)\n    ) ENGINE=InnoDB\n'

In [627]:
# 
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")



Creating table businesses: already exists.
Creating table reviews: already exists.


In [628]:
#close connection

cursor.close() 

cnx.close()

## Part 2: Create ETL pipeline for the business data from the API

In [629]:
import requests
client_id = '00DmQSvccbT6M4gvjIofeg'
api_key = 'eHvn2Iiz4ddoAbrxe7ORZGcoKsVbIxjuJMvQz9tdFy0v7OVNsNMLl0J4G0D2vKOlcuZcBNFUjgb19plwZJElAtQVWtbN6aXH9r2hQ9OedhxVIRekSvuJiiC-ucmxXXYx'

In [630]:
# Write a function to make a call to the yelp API
term = 'Outdoor Concert Venues'
location = 'New York NY'
category = ('price',['businesses'])
SEARCH_LIMIT = 50
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "senX8w2mEBIjurVgmHRt0g", "alias": "pier-63-at-hudson-river-park-new-york", "name": "Pier 63 at Hudson River Park", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/Tb32g4C8nj6d8G0T6-W2KA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/pier-63-at-hudson-river-park-new-york?adjust_creative=00DmQSvccbT6M4gvjIofeg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=00DmQSvccbT6M4gvjIofeg", "review_count": 9, "categories": [{"alias": "parks", "title": "Parks"}, {"alias": "musicvenues", "title": "Music Venues"}], "rating": 4.5, "coordinates": {"latitude": 40.743097, "longitude": -73.99333}, "transactions": [], "price": "$", "location": {"address1": "W 23rd St", "address2": "", "address3": "", "city": "New York", "zip_code": "10011", "country": "US", "state": "NY", "display_address": ["W 23rd St", "New York, NY 10011"]}, "phone": "+12129896363", "display_phone": "(212) 989-6363", "distance": 4187.508571

In [631]:
response.text

'{"businesses": [{"id": "senX8w2mEBIjurVgmHRt0g", "alias": "pier-63-at-hudson-river-park-new-york", "name": "Pier 63 at Hudson River Park", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/Tb32g4C8nj6d8G0T6-W2KA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/pier-63-at-hudson-river-park-new-york?adjust_creative=00DmQSvccbT6M4gvjIofeg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=00DmQSvccbT6M4gvjIofeg", "review_count": 9, "categories": [{"alias": "parks", "title": "Parks"}, {"alias": "musicvenues", "title": "Music Venues"}], "rating": 4.5, "coordinates": {"latitude": 40.743097, "longitude": -73.99333}, "transactions": [], "price": "$", "location": {"address1": "W 23rd St", "address2": "", "address3": "", "city": "New York", "zip_code": "10011", "country": "US", "state": "NY", "display_address": ["W 23rd St", "New York, NY 10011"]}, "phone": "+12129896363", "display_phone": "(212) 989-6363", "distance": 4187.508571127495}, {"id": "Isq7hk-ywXeAh

In [632]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [633]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [634]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n') #Seperate out data

businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





In [635]:
# Write a function to take your parsed data and insert it into the DB

In [636]:
yelp_data =response.json()

In [637]:
yelp_data['businesses'][:]

[{'id': 'senX8w2mEBIjurVgmHRt0g',
  'alias': 'pier-63-at-hudson-river-park-new-york',
  'name': 'Pier 63 at Hudson River Park',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Tb32g4C8nj6d8G0T6-W2KA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/pier-63-at-hudson-river-park-new-york?adjust_creative=00DmQSvccbT6M4gvjIofeg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=00DmQSvccbT6M4gvjIofeg',
  'review_count': 9,
  'categories': [{'alias': 'parks', 'title': 'Parks'},
   {'alias': 'musicvenues', 'title': 'Music Venues'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.743097, 'longitude': -73.99333},
  'transactions': [],
  'price': '$',
  'location': {'address1': 'W 23rd St',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10011',
   'country': 'US',
   'state': 'NY',
   'display_address': ['W 23rd St', 'New York, NY 10011']},
  'phone': '+12129896363',
  'display_phone': '(212) 989-6363',
  'distance': 418

In [638]:
for venue in yelp_data['businesses'][:]:
    if len(venue.keys()) < 16:
        venue['price'] = ''
        continue
    
    
    
# missing_keys = (id.keys()) - (price.keys())
# missing_keys
# for k in missing_keys:
#     dict1[k] = dict2[k]

In [639]:
# Write a script that combines the three functions above into a single process.
for venue in yelp_data['businesses']:
    venue_info= {}
    venue_info['id']= venue['id']
    venue_info['name']= venue['name']
    venue_info['rating']= venue['rating']
    venue_info['price']= venue['price']
 




venue_info

{'id': 'hKChLzaG8tyrSrg3U_wRNw',
 'name': 'Threes Brewing',
 'rating': 4.0,
 'price': '$$'}

In [640]:
venue_list = []

for venue in yelp_data['businesses']:
    
        
    venue_tuple = (venue['id'], venue['name'],
                   venue['rating'], venue['price'])
    venue_list.append(venue_tuple)

venue_list


[('senX8w2mEBIjurVgmHRt0g', 'Pier 63 at Hudson River Park', 4.5, '$'),
 ('Isq7hk-ywXeAhUOIpwIeMg', 'Pier 17', 4.0, '$$'),
 ('v6iS47Bu4S7NfQN9zTEgYg', 'MCU Park', 4.0, ''),
 ('Qy-GX1fnZ9tMq6Iuw1Gsfg', 'New World Stages', 4.0, ''),
 ('S3T5lIRgXv2zZF964XjIiA', 'Terminal 5', 3.0, '$$'),
 ('Cmc4FRaSNKxXMSubgwhcTg', 'City Parks Foundation SummerStage', 4.0, ''),
 ('epBbIB9tE0JHhQ2eMOJNNQ', 'Piers 92/94', 3.0, ''),
 ('gkx8nMeISi_kfon-FkKz9Q', 'Kings Theatre', 4.0, '$$'),
 ('spQMbvAokpUrgrDCSfYSgw', 'Citi Field', 4.0, ''),
 ('o-F89VaYJTMjLnRTVM_6jQ', 'Brooklyn Steel', 3.5, '$$'),
 ('jq4AJMB31X4gzDj0Vw2k2A', 'Irving Plaza', 3.5, '$$'),
 ('rc-KlFsCOrvlugCJisEM-g', 'Bronx Museum of the Arts', 4.0, ''),
 ('hU8toaysloKHX2zOUDHlHg',
  'USTA Billie Jean King National Tennis Center',
  4.0,
  ''),
 ('Y_JN78je9-jdqlPF9IxF8w', 'JBL Live at Pier 97', 3.5, '$$'),
 ('JOr11q2DKWezsDNTTTtS8A', 'Barclays Center', 3.5, '$$$'),
 ('Ecf8YEqNzom53Y93Ytpi8g', 'Madison Square Garden', 4.0, ''),
 ('XbuE5QykVIvykvkLdO

In [650]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'yelp'
)
cursor = cnx.cursor()

In [656]:
insert_query ="""INSERT into businesses (id,name,rating,price)
                 Values(%s,%s,%s,%s)"""

In [653]:
cursor.executemany(insert_query, venue_list)

IntegrityError: 1062 (23000): Duplicate entry 'senX8w2mEBIjurVgmHRt0g' for key 'PRIMARY'

In [654]:
cnx.commit()

In [655]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

 

stmt = """INSERT into businesses (id,name,rating,price,review_count)
                 Values(%s,%s,%s,%s,%s)"""
cursor.executemany(stmt, venue_list) 

InterfaceError: Failed executing the operation; Not enough parameters for the SQL statement

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [526]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant


NameError: name 'businesses' is not defined

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)